In [5]:
import pandas as pd
import gzip
from Bio import SeqIO
import regex

read_1_regex = regex.compile(open("../results_CH65/regexes/read_1_regex.txt").read())
read_2_regex = regex.compile(open("../results_CH65/regexes/read_2_regex.txt").read())

max_reads_to_consider = 1000000000
    
total_reads_considered = 0
reads_matching_pattern = 0
parsed_reads = []

# Go through both input fastq files simultaneously line by line
with gzip.open('../results_CH65/grouped_fastq/CH65_a_0_1_R1.fastq.gz', 'rt') as read_1_handle:
    with gzip.open('../results_CH65/grouped_fastq/CH65_a_0_1_R2.fastq.gz', 'rt') as read_2_handle:
        for (read_1, read_2) in zip(
                    SeqIO.parse(read_1_handle, 'fastq'),
                    SeqIO.parse(read_2_handle, 'fastq')):
            if total_reads_considered >= max_reads_to_consider:
                break
            total_reads_considered += 1

            match_1 = read_1_regex.match(str(read_1.seq))
            match_2 = read_2_regex.match(str(read_2.seq))

            if match_1 is None or match_2 is None:
                continue
            reads_matching_pattern += 1

            UMI_1 = match_1.group('UMI')
            inline_idx_1 = match_1.group('index')
            read_1_seq = match_1.group('read_seq')

            UMI_2 = match_2.group('UMI')
            inline_idx_2 = match_2.group('index')
            read_2_seq = match_2.group('read_seq')

            parsed_reads.append((inline_idx_1, inline_idx_2, UMI_1, UMI_2, read_1_seq, read_2_seq))



In [6]:
parsed_reads

[('ATCGCAGGCAT',
  'TCGGTGGTACG',
  'GTGATACG',
  'AAGGCAGG',
  'CTGGCCAGACGGCTAGAATTACCTGCGGCGGTAACGATATTGGACGGAAAAGTGTGCACTGGTACCAGCAGAAGCCAGGCCAGGCCCCTGTGCTGGTCGTCTATGATGATAGCGACAGACCCTCAGGCATACCTGAGCGATTCTCTGGCTCCACCTCTGGGAACACGGCCACCCTGACCATCAGCAGGGTCGACGCCGGGGAGGAGGCCGACTCTTCCGGTCAGGAGGGGGA',
  'CCTCCACGGGCACAGTAATACACGGCCGTGTCGTCAGATCTCAGTCCATTGACCTCCATGTAGGCTGTGCTGATGGACGTGTCCCTGGTCATGGTGACCCAGCCCTGAAACTTCTGTGCATAGTTTCTATCACCACTGTTAGGATGGAACCTTCCCATCCACTCGAGCCCTTGTCCAGGGGCTTGTCGCACCCATGGCATGAGGGCATCGGTGATGGTCGATCCAGACGCGG'),
 ('ATCGCAGGCAT',
  'TCGGTGGTACG',
  'GGGAAATT',
  'CAATGTAT',
  'CTGGCCAGACGGCTAGAATTACCTGCGGCGGTAACGATATTGGAAGTAAAAGTGTGCACTGGAATCAGCAGAAGCCAGGCCAGGTCCCTGTGCTGGTCGTCTATGATGATAGCGACAGACCCTCAGGCATACCTGAGCGATTCTCTGGCTCCAACTCTGGGAACACGGCCACCCTGACCATCAGCAGGGTCGAAGCCGGGGATGAGGCCGACTATTACTGTCAGGTGTGGGA',
  'CCTCCACGGGCACAGTAATACACGGCCGTGTCGACAGATCTCAGCCTGCTGACCTCCATGTAGGCTGAGCTGATGGACGTGTCCCTGGTCATGGTGACCCAGCCCTGAAACTTCTGTGCATAGATTGAGCCACCACTCATAGGGTTCATCCATCCCAT

In [ ]:
import pandas as pd
import gzip
from Bio import SeqIO
import regex
import yaml

read_1_regex = regex.compile(open("../results_CH65/regexes/read_1_regex.txt").read())
read_2_regex = regex.compile(open("../results_CH65/regexes/read_2_regex.txt").read())

config = yaml.safe_load(open('../metadata/config.yaml'))

UMI_length = config['umi_length']
col_inline_idx = config['col_inline_indices']
row_inline_idx = config['row_inline_indices']

total_reads_considered = 0
reads_matching_pattern = 0
parsed_reads2 = []

# Go through both input fastq files simultaneously line by line
#with open(snakemake.output.tsv, 'w') as fw:
#     fw.write("\t".join(
#                     ["col_idx", "row_idx", "UMI_1", "UMI_2",
#                      "read_1", "read_2"]))
with gzip.open('../results_CH65/grouped_fastq/CH65_a_0_1_R1.fastq.gz', 'rt') as read_1_handle:
    with gzip.open('../results_CH65/grouped_fastq/CH65_a_0_1_R2.fastq.gz', 'rt') as read_2_handle:
        for (read_1, read_2) in zip(
                    SeqIO.parse(read_1_handle, 'fastq'),
                    SeqIO.parse(read_2_handle, 'fastq')):
            #if snakemake.config['debug_mode'] and total_reads_considered >= snakemake.config['max_reads_per_file_if_debugging']:
            #    break
            total_reads_considered += 1
            read_1, read_2 = read_1.seq, read_2.seq
            UMI_1 = read_1[:UMI_length]
            inline_idx_1 = None
            # read 1 use col index
            for idx in col_inline_idx:
                if read_1[UMI_length:].startswith(idx):
                    inline_idx_1 = idx
                    break
            if inline_idx_1 is None:
                continue
            read_1_seq = read_1[UMI_length + len(inline_idx_1):]
            
            UMI_2 = read_2[:UMI_length]
            inline_idx_2 = None
            # read 1 use col index
            for idx in row_inline_idx:
                if read_2[UMI_length:].startswith(idx):
                    inline_idx_2 = idx
                    break
            if inline_idx_2 is None:
                continue
            read_2_seq = read_2[UMI_length + len(inline_idx_2):]
            
            reads_matching_pattern += 1
            parsed_reads2.append((inline_idx_1, inline_idx_2, UMI_1, UMI_2, read_1_seq, read_2_seq))
#             fw.write("\t".join(
#                 [inline_idx_1, inline_idx_2, UMI_1, 
#                  UMI_2, read_1_seq, read_2_seq]
#             ))
#                 
# # Write stats
# (pd.DataFrame({
#     'sample' : [snakemake.wildcards.sample],
#     'fastq_file' : [snakemake.input.read_1_fastq],
#     'total_reads_considered': [total_reads_considered],
#     'reads_matching_pattern': [reads_matching_pattern], 
#     'fraction_of_fastq_accepted' : [reads_matching_pattern / total_reads_considered]
# })
#  .set_index('sample')
#  .to_csv(snakemake.output.stats, sep='\t')
# )

In [ ]:
parsed_read1 == parsed_read2 